## Что такое обучение?

Компьютеры считывают данные, как мы видели в блокнотах 1 и 2. Затем мы можем создавать функции, которые моделируют эти данные для принятия решений, как мы видели в блокнотах 3 и 5. Но как убедиться, что модель действительно соответствует данным?

В последнем блокноте мы увидели, что мы можем поиграться с параметрами нашей функции, определяющими модель, чтобы уменьшить функцию потерь. Однако мы не хотим сами выбирать параметры модели. Самостоятельный выбор параметров работает *достаточно хорошо*, когда мы имеем простую модель и всего несколько точек данных, но для более подробных моделей и больших наборов данных такой номер не пройдёт. Вместо этого мы хотим, чтобы наша машина *изучала* параметры, которые соответствуют модели, без необходимости самим манипулировать параметрами. В этом блокноте мы поговорим об «обучении» в **ML**.

### Мотивация: Подгонка параметров вручную

Давайте вернемся к нашему примеру подбора параметров из блокнота 3. Напомним, что мы смотрели на то, может ли количество зеленого на фотографиях различать яблоко и банан, и использовали сигмовидную функцию для моделирования нашего выбора «яблоко или банан». используя количество зеленого цвета в изображении.

In [ ]:
using Plots; gr()
using Images; using Interact

σ(x,w,b) = 1 / (1 + exp(-w*x+b))

apple =  load("data/10_100.jpg")
banana = load("data/104_100.jpg")
apple_green_amount =  mean(Float64.(green.(apple)))
banana_green_amount = mean(Float64.(green.(banana)));

@manipulate for w in -10:0.01:30, b in 0:0.1:30
    
    plot(x->σ(x,w,b), 0, 1, label="Model", legend = :topleft, lw=3)
    scatter!([apple_green_amount],  [0.0], label="Apple")
    scatter!([banana_green_amount], [1.0], label="Banana")
    
end

Интуитивно понятно, как вы настраивали ползунки так, чтобы модель отправляла яблоки в 0 и бананы в 1? Скорее всего, вы сделали следующее:

**Немного переместить ползунки, посмотрить, движется ли кривая в правильном направлении, и если это так, продолжать.**

Для машины «обучение» - это тот же процесс, переведенный в математику!

## «Обучение подталкиванием»: процесс спуска

Давайте начнем формализовать эту идею. Чтобы подтолкнуть кривую в «правильном направлении», нам нужно измерить «насколько правильна» и «насколько неправильна» модель. Когда мы переводим идею «правильного направления» в математику, мы получаем **функцию потерь**, `L (w, b)`, как мы видели в блокноте 5. 

Мы говорим, что функция достигает минимума, когда модель `σ(x, w, b)` работает лучше всего. Теперь мы хотим создать функцию потерь, которая будет минимизирована, когда яблоко находится в точке «0», а банан в точке «1». Если данные (количество зеленого цвета) для нашего яблока равны $ x_1 $, то наша модель выведет $ σ (x_1, w, b) $ для нашего яблока.

Итак, мы хотим, чтобы разница $ 0 - σ (x_1, w, b) $ была небольшой. Точно так же, если наши данные для нашего банана (количество зеленого банана) составляет $ x_2 $, мы хотим, чтобы разница в $ 1 - σ (x_2, w, b) $ была небольшой. Чтобы создать нашу функцию потерь, давайте сложим квадраты разности выходных данных модели от желаемых выходных данных для яблока и банана. Мы получаем

$$ L(w,b) = (0 - σ(x_1, w, b))^2 + (1 - σ(x_2, w, b))^2. $$

$L(w, b)$ имеет минимум когда достигает `0` для яблока и` 1` для банана, и, таким образом, стоимость является самой низкой, когда модель «верна». 

Мы можем визуализировать эту функцию, построив ее в 3D с помощью функции «поверхность».

In [ ]:
# plotly()
gr()

In [ ]:
L(w, b) = (0 - σ(apple_green_amount,w,b))^2 + (1 - σ(banana_green_amount,w,b))^2

w_range = 10:0.1:13
b_range = 0:1:20

L_values = [L(w,b) for b in b_range, w in w_range]


@manipulate for w in w_range, b in b_range
    p1 = surface(w_range, b_range, L_values, xlabel="w", ylabel="b", cam=(70,40), cbar=false, leg=false)
    scatter!(p1, [w], [b], [L(w,b)+1e-2], markersize=5, color = :blue)
    p2 = plot(x->σ(x,w,b), 0, 1, label="Model", legend = :topleft, lw=3)
    scatter!(p2, [apple_green_amount],  [0.0], label="Apple", markersize=10)
    scatter!(p2, [banana_green_amount], [1.0], label="Banana", markersize=10, xlim=(0,1), ylim=(0,1))
    plot(p1, p2, layout=(2,1))
end

Синий шар на трехмерном графике показывает текущий выбор параметров, обозначенный как `(w, b)`. Ниже 3D-графика показан 2D-график соответствующей модели с этими параметрами. Обратите внимание, что когда синий шар катится по склону, модель становится лучше. Наша функция потерь дает нам математическое представление о «холме», а процесс «обучения подталкиванием» просто катит мяч вниз по этому холму.

Чтобы сделать это математически, нам нужно знать, направление «спуска». Вспомните из матана или физики, что производная от `L` по отношению к` b` говорит вам, как изменяется `L` при изменении` b`. Таким образом, чтобы скатиться вниз, мы должны идти в направлении, где производная отрицательна (функция понижается) для каждого параметра. Этот спуск есть движение по направлению убывания **градиента**, $ \nabla L $. Это означает, что «метод обучения с помощью подталкивания» можно перефразировать в математических терминах следующим образом:

1. Рассчитать градиент
2. Немного переместиться в направлении отрицательного градиента
3. Повторить

Этот процесс катания шара в направлении отрицательного градиента называется **градиентным спуском**; а по математически:

$$p_{n+1} = p_n - \eta \nabla L(p_n).$$

Здесь $ p_n $ представляет вектор текущих параметров $ (w, b) $; $ \nabla L (p_n) $ - градиент функции потерь с учетом этих параметров. Мы начинаем с $ p_n $ и меняем его на $ \eta \nabla L (p_n) $, где $ \eta $ - это маленький размер шага, который определяет, как далеко мы перемещаем параметры в направлении отрицательного градиента; Заметьте, что если вы зайдете слишком далеко, вы перешагнете минимум! В результате получается $ p_ {n + 1} $, новый вектор параметров.

![image.png](data\grad.gif)

Если мы повторим этот процесс, то получим параметры, в которых модель правильно помечает яблоки как `0`, а бананы как` 1`. Когда это происходит, модель извлекла уроки из данных и затем может прочитать фотографии и сказать вам, являются ли они яблоками или бананами!

#### Упражнение 1

Используйте следующие термины, чтобы заполнить предложения ниже. Термины могут использоваться более одного раза или не использоваться вообще:
> градиент, функция потерь, производная, градиентный спуск, изучение.

* Мы можем думать о _ (A) _ как о 1D версии _ (B) _.
* Мы можем визуализировать _ (C) _ как холм.
* В приведенном выше объяснении движение вниз по склону называется _ (D) _ и означает путешествие по _ (E) _.
* Чтобы определить правильность модели, мы используем _ (F) _.
* Когда наша программа может минимизировать _ (G) _ самостоятельно, мы говорим, что это _ (H) _.

<br><br>

A)<br>
B)<br>
C)<br>
D)<br>
E)<br>
F)<br>
G)<br>
H)<br>